In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [3]:
def rmse(y_pred, y_true):
    return np.sqrt(((y_pred - y_true) ** 2).mean())

In [4]:
t = 1
def showPlot(true, prediction, model , test_set, size='M', save=False):
    if(size=='L'):
        plt.figure(figsize=(28, 10))
    plt.plot(true, color = 'red', label = 'Real')
    plt.plot(prediction, color = 'blue', label = 'Prediction')
    plt.title('Industry Elec Prediction' + model + ' ' + test_set + ' t' + str(t))
    plt.xlabel('Time')
    plt.ylabel('kW')
    plt.legend()
    if(save):
        plt.savefig('Image/' + model + '-' + test_set + '-' + str(t) + '.jpeg')
    plt.show()

In [5]:
def augFeatures(data):
    features_select = ["IndustryElec(kW)", "isHoliday", "isHoliday(Next7)"]
    data_set = pd.DataFrame(data[features_select])
    return data_set
def normalize(train):
    sc = MinMaxScaler(feature_range = (0, 1))
    training_set_scaled = sc.fit_transform(train)
    return sc, training_set_scaled 

In [6]:
def buildTest(startDate, sc, pastDay, futureDay, pre_step):
    dataset_total = augFeatures(df)
    dataset_total = dataset_total[(startDate-pastDay):(startDate-pastDay)+(pre_step*pastDay)]
    inputs = dataset_total.values
    inputs = inputs.reshape(-1, dataset_total.shape[1])
    inputs = sc.transform(inputs)
    X_test = []
    for i in range(0, pre_step):
        X_test.append(inputs[pastDay*i:pastDay*(i+1)])
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))  

    return X_test

In [7]:
from sklearn.preprocessing import MinMaxScaler
def data():
    def buildTrain(train, pastDay=7, futureDay=7):
        X_train, Y_train = [], []
        for i in range(pastDay, len(train)-futureDay):
            X_train.append(train[i-pastDay:i])
            Y_train.append(train[i:i+futureDay, 0])
        X_train, Y_train = np.array(X_train), np.array(Y_train)  
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
        return X_train, Y_train
    def splitData(X, Y, rate):
        X_train = X[int(X.shape[0]*rate):]
        Y_train = Y[int(Y.shape[0]*rate):]
        X_val = X[:int(X.shape[0]*rate)]
        Y_val = Y[:int(Y.shape[0]*rate)]
        return X_train, Y_train, X_val, Y_val
    def augFeatures(data):
        features_select = ["IndustryElec(kW)", "isHoliday", "isHoliday(Next7)"]
        data_set = pd.DataFrame(data[features_select])
        return data_set
    def normalize(train):
        sc = MinMaxScaler(feature_range = (0, 1))
        training_set_scaled = sc.fit_transform(train)
        return sc, training_set_scaled 
    df = pd.read_csv('Data/elec_merge_20160101_20190930(en).csv')
    RawTrain = df[366:1096]
    RawTest = df[1096:]
    training_set = augFeatures(RawTrain)
    sc, training_set_scaled = normalize(training_set)

    Timesteps = 7
    OutputDay = 7
    X_train, Y_train = buildTrain(training_set_scaled, Timesteps, OutputDay)
    
    X, Y, Xv, Yv= splitData(X_train, Y_train, 0.1)
    
    return X, Y, Xv, Yv

In [8]:
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

Using TensorFlow backend.


In [9]:
from keras.models import Sequential
from keras.layers import Dense, TimeDistributed, Flatten
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import CuDNNLSTM
from keras.layers import SimpleRNN
from keras.layers import Dropout
from keras.layers import BatchNormalization, Bidirectional
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers

In [10]:
def create_model(X, Y, Xv, Yv):
    model = Sequential()
    model.add(CuDNNLSTM(units = 256, return_sequences = True, input_shape = (X.shape[1], X.shape[2])))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Flatten())
    model.add(Dense(units = 7))
    
    optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(optimizer = optimizer, loss = 'mean_squared_error')
    
    model.summary()
    model.fit(X, Y,
              batch_size={{choice([16, 32, 64])}},
              epochs={{choice([100, 200, 400])}},
              validation_data=(Xv, Yv),
              verbose=0)
    
    return {'loss': min(model.history.history['val_loss']), 'status': STATUS_OK, 'model': model}

In [11]:
X, Y, Xv, Yv = data()

best_run, best_model = optim.minimize(model=create_model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=5,
                                      trials=Trials(),
                                      verbose=False,
                                      notebook_name='IndustryElec_(LSTM)_Hyperas')

print("Evalutation of best performing model:")
print(best_model.evaluate(Xv, Yv))
print(best_run)

UnicodeDecodeError: 'cp950' codec can't decode byte 0xe2 in position 12971: illegal multibyte sequence

In [ ]:
best_model.save('best_model_2.h5')

In [ ]:
df = pd.read_csv('Data/elec_merge_20160101_20190930(en).csv')
RawTrain = df[366:1096]
RawTest = df[1096:]
training_set = augFeatures(RawTrain)
sc, training_set_scaled = normalize(training_set)

Timesteps = 7
OutputDay = 7

In [ ]:
Pre_step = 39

In [ ]:
pre_time = 7 * Pre_step
test_set = pd.DataFrame(RawTest[["IndustryElec(kW)"]])
test_set = test_set[:pre_time]
real_elec = test_set.values

X_test = buildTest(1096, sc, Timesteps, OutputDay, Pre_step)

In [ ]:
test_predict = best_model.predict(X_test)
test_predict = test_predict * (sc.data_max_[0]-sc.data_min_[0]) + sc.data_min_[0]

test_prediction = []
for i in range(Pre_step):
    test_prediction= np.concatenate((test_prediction, test_predict[i]), axis=0)
test_prediction = np.reshape(test_prediction, (real_elec.shape[0], real_elec.shape[1]))

In [ ]:
print("RMSE: ", rmse(real_elec, test_prediction))
print("mape: ", mean_absolute_percentage_error(real_elec, test_prediction))
showPlot(real_elec, test_prediction, "ElecHyperas_1" , str(pre_time), save=False)